In [ ]:
from obspy import read
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import json
import calendar

In [ ]:
def make_plot_dictionary():
    """
    Makes dictionary with available plots in directories.

    plot_dictionary[year][month][day][doy] is the corresponding day of the year for day and month of year,
    plot_dictionary[year][month][day][stations] are the stations that have plots for day and month of year.

    :return: plot dictionary
    """
    plot_dict = {}
    for year in [1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977]:
        plot_dict[year] = {}

        for month in range(1, 13):
            plot_dict[year][month] = {}

        # check if each station has a plot for a day
        for station in ["s11", "s12", "s14", "s15", "s16"]:
            directory = os.path.join(os.getcwd(), "plots", station, str(year))
            available_days = []
            for (dirpath, dirnames, filenames) in os.walk(directory):
                available_days.extend(dirnames)
                break
            available_days = set([int(day) for day in available_days])

            if calendar.isleap(year):
                day_range = range(1, 367)
            else:
                day_range = range(1, 366)
            # days of year
            for day in day_range:
                # days of year converted to dates
                date = pd.to_datetime(year*1000 + day, format="%Y%j")
                day_month = int(date.day)
                month = int(date.month)

                # first time visiting day of month, init
                if day_month not in plot_dict[year][month]:
                    plot_dict[year][month][day_month] = {}
                    plot_dict[year][month][day_month]["doy"] = "%03d" % day
                    plot_dict[year][month][day_month]["stations"] = []

                # station has a plot for day
                if day in available_days:
                    plot_dict[year][month][day_month]["stations"].append(station)

        # output
        file_path_out = os.path.join(os.getcwd(), "plots", "plot_dict.json")
        print(file_path_out)
        with open(file_path_out, "w") as file_out:
            file_out.write(json.dumps(plot_dict, indent=4))

In [ ]:
make_plot_dictionary()

In [ ]:
def get_saved_stations(return_num=False):
    """
    Gets available stations from saved directories.

    :param return_num: if True returns station name as number instead of string (e.g. 11 instead of "s11")
    :return: station name list (empty if data doesn't exist)
    """
    directory = os.path.join(os.getcwd(), "data")
    if not os.path.isdir(directory):
        return []

    stations = []
    for (dirpath, dirnames, filenames) in os.walk(directory):
        stations.extend(dirnames)
        break

    try:
        stations.remove("nakamura")
    except ValueError:
        pass

    if return_num:
        stations = [int(station[1:]) for station in stations]

    return stations

In [ ]:
def get_saved_years(station, return_num=False):
    """
    Gets available data years for a station from saved directories.

    :param station: station name
    :param return_num: if True returns year as number instead of string
    :return: years list (empty if station name is invalid or data doesn't exist)
    """
    # add "s" in front of station number
    station = str(station)
    if not station.startswith("s"):
        station = "s" + station

    directory = os.path.join(os.getcwd(), "data", station)
    if not os.path.isdir(directory):
        return []

    years = []
    for (dirpath, dirnames, filenames) in os.walk(directory):
        years.extend(dirnames)
        break

    if return_num:
        years = [int(year) for year in years]

    return years

In [ ]:
def sensor_plot(ax=None, fontsize=15, title='', tr=None):
    """
    Adds plot with PSE data to axis.

    :param ax: axis
    :param fontsize: plot title fontsize
    :param title: plot title
    :param tr: trace
    """
    ax.locator_params(nbins=3)
    ax.set_xlabel('Time', fontsize=fontsize)
    ax.set_ylabel('DU', fontsize=fontsize)
    ax.set_yticks(np.linspace(min(tr.data), max(tr.data), 5))
    ax.set_title(title, fontsize=20, fontweight="bold", pad=10)
    ax.plot(tr.times("matplotlib"), tr.data, "b-")
    ax.xaxis_date()

In [ ]:
if __name__ == "__main__":
    plot_data("s11")
    plot_data("s12")
    plot_data("s14")
    plot_data("s15")
    plot_data("s16")

    make_plot_dictionary()

In [ ]:
import obspy
st = obspy.read("xa.s14.00.mh2.1972.004.0.mseed")
print(st)
st.plot()


In [1]:
import pandas as pd
df=pd.read_csv("Quakes locations.csv")
df

,Year,Day,H,M,S,Lat,Long,Magnitude,Comments,Date
0,1971,107,7,0,55,48,35,2.8,NaN,Sat- 17\April\1971
1,1971,140,17,25,10,42,-24,2.0,NaN,Thu- 20\May\1971
2,1971,192,13,24,45,43,-47,1.9,Or lat -42 long -60,Sun- 11\July\1971
3,1972,2,22,29,40,54,101,1.9,NaN,Sun- 02\January\1971
4,1972,261,14,35,55,12,46,1.0,NaN,Sun- 17\September\1972
5,1972,341,23,8,20,51,45,1.4,NaN,Wed- 06\December\1972
6,1972,344,3,50,15,-20,-80,1.2,NaN,Sat- 09\December\1972
7,1973,39,22,52,10,33,35,0.8,NaN,Thu- 08\Febraury\1973
8,1973,72,7,56,30,-84,-134,3.2,NaN,Tue- 13\March\1973
9,1973,171,20,22,0,-1,-71,2.2,NaN,Wed- 20\June\1973


In [ ]:
A=df.groupby(["Year"])
A["Day"].value_counts().index